# Hidden symmetry 
## The motivation
* Go beyond the integeralbility 
* Exploring the new structure of non-plannar amplitude
* Understanding the gauge/gravity duality deeply 

## The expected properties of the symmetry 
* Leave the amplitude invariant 
* The hidden symmetry is a global symmetry
* There are some factors which related to the ordering in the kinematic algebra when symmetry act on the external legs 
* The structure constant of the algebra should be the BCJ numerator 
* It should be Yangian symmetry for the planar amplitude
* It have martix representation in the spinor space

# The function of the algebra product  

In [3]:
using vectorDot

ArgumentError: ArgumentError: Package vectorDot not found in current path:
- Run `import Pkg; Pkg.add("vectorDot")` to install the vectorDot package.


# BCJ relations and the map from DDM basis to minimal basis

In [2]:
a=1

1

In [1]:
sx=1

1

# Ansatz of the symmetry action on the external legs

# Equations from the requirement of the amplitude invariant.